# logit-регрессия: LR-тест, совместная значимость

In [27]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import chi2 # критические значения chi2-распределения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
# импорт датасета
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))
# импорт данных из локального файла
# df = pd.read_csv('loanapp.csv')
df

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,89,1,1120,0,72,0,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.754237,0.0,1
1,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
2,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
3,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
4,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,1,158,1,1120,0,96,0,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.897727,0.0,1
1985,1,35,1,1120,0,169,1,1.0,1.0,0.0,...,1,1,0,0,0,1,0.0,0.111111,0.0,1
1986,2,225,1,1120,0,49,0,2.0,1.0,0.0,...,1,0,1,0,0,1,1.0,1.000000,0.0,1
1987,1,98,1,1120,1,110,1,1.0,0.0,0.0,...,1,1,0,0,0,1,0.0,0.455814,0.0,1


## Спецификация и подгонка модели
Для датасета `loanapp`
рассморим logit-регрессию **approve на appinc, appinc^2, mortno, unem, dep, male, married, yjob, self**

In [29]:
mod = smf.logit(formula='approve~appinc+I(appinc**2)+mortno+unem+dep+male+married+yjob+self', data=df) # спецификация модели
res = mod.fit() # подгонка модели
res.nobs # число наблюдений, на которых была подогнана модель

Optimization terminated successfully.
         Current function value: 0.362116
         Iterations 7


1971

## LR-тест: совместная значимость
Тестируем значимость влияния дохода, т.е. $$H_0:\beta_{appinc}=\beta_{appinc^2}=0.$$

Тестовая статистика $LR=2(\log L-\log L_{r})$ где 
- $\log L$ – лог-правдоподобие для исходной регрессии
- $\log L_{r}$ - лог-правдоподобие для регрессии с ограничениями (без переменных из гипотезы)

Для вычисления тестовой статистики нужно оценить две вложенных модели: без ограничений и с ограничениями

## <span style="color:#59afe1">**Важно**: обе регрессия д.б. оценены на одном и том же датасете! </span>

## Уровень значимости
Пусть уровень значимости $\alpha=1\%=0.01$

In [30]:
# подгонка модели с ограничениями
df_mod = df[['approve','appinc','mortno','unem','dep', 'male', 'married', 'yjob', 'self']].dropna()
mod_r = smf.logit(formula ='approve~mortno+unem+dep+male+married+yjob+self', data = df_mod)
res_r = mod_r.fit()
res_r.nobs # число наблюдений, на которых была подогнана модель

Optimization terminated successfully.
         Current function value: 0.363993
         Iterations 7


1971

### Тестовая статистика LR-теста 

In [31]:
# Тестовая статистика LR-теста с округленим
lr_stat=2*(res.llf-res_r.llf)
lr_stat.round(3)

7.399

### Степени свободы для $\chi^2$-распределения 

In [32]:
res.df_model-res_r.df_model

2.0

### P-значение тестовой статистики LR-теста

In [33]:
# P-значение тестовой статистики LR-теста с округленим
lr_pvalue = chi2.sf(lr_stat, df=res.df_model-res_r.df_model)
lr_pvalue.round(3)

0.025

### Критическое значение $\chi_{df}^2(\alpha)$

In [34]:
sign_level = 0.01 # уровень значимости
chi2.ppf(q=1-sign_level, df=res.df_model-res_r.df_model).round(3) 

9.21

### Значимо ли влияние дохода?

`влияние дохода незначимо` $(LR<\chi_{df}^2(\alpha))$

In [35]:
# Статистика теста Вальда и её P-значение
res.wald_test('appinc=I(appinc ** 2)=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[7.1202905]], p-value=0.028434694253215525, df_denom=2>